In [20]:
from lsh_helpers import *
import networkx as nx
import matplotlib.pyplot as plt
import random

In [35]:
def build_schema(lshensemble, index_dict):
    sgraph = nx.DiGraph()
    edge_labels={}
    # loop through index dict and search for containment relationships, if found, 
    # add an edge between those 2 tables with edge label as the "join" condition
    for k,v in index_dict.items():
#         ftable_label = v['table'].replace('_','\_')
#         fcolumn_label = v['col'].replace('_','\_')
        try:
            candidates = search_index(lshensemble, index_dict, v['table'], v['col'])
#             print(f"candidates for {v['table']}:{v['col']}")
#             print(candidates)
#             print('\n')
        except ZeroDivisionError:
            continue
        if(candidates):
            for c in candidates:
                ctable, ccol = c.split('->')
    #             ptable_label = ctable.replace('_','\_')
    #             pcolumn_label = ccol.replace('_','\_')
                if(ctable==v['table']):
                    continue
                rnum = random.random()
                if(rnum<0.0005):
                    sgraph.add_edge(v['table'], ctable)
                    edge_labels[(v['table'], ctable)] = \
                    f"{v['table']}.{v['col']}---{ctable}.{ccol}"
    
    return edge_labels, sgraph

#     pos=nx.spring_layout(sgraph)
#     nx.draw(
#     sgraph, pos, edge_color='black', width=1, linewidths=1,
#     node_size=500, node_color='pink', alpha=0.9,
#     labels={node: node for node in sgraph.nodes()}
# )
#     nx.draw_networkx_edge_labels(
#     sgraph, pos,
#     edge_labels=edge_labels,
#     font_color='red'
# )


In [40]:
def draw_graph(sgraph):
    
    pos = nx.random_layout(sgraph, seed=0)
    nx.draw_random(sgraph, node_color='pink', 
                   width=0.1, node_size=20,font_size=2,
            with_labels=True, edge_cmap=plt.cm.YlOrBr)
    
    plt.figure(1, figsize=(1500, 1500), dpi=1000)
    plt.savefig(f'sgraph.png',dpi=1000)
    
#     return edge_labels

In [31]:
# create a graph structure based on containment threshold

In [24]:
if __name__ =='__main__':
    conn = psycopg2.connect("dbname=curation_data_lake user=postgres")
#     conn = psycopg2.connect("dbname=test_hash_db user=postgres")
    conn.autocommit=True
#     create_and_serialize_index(conn, 'april16hashes',num_perm=256, thresh=0.8)

In [25]:
lshensemble, index_dict = load_index(conn, index_table='april16hashes', threshold=0.9,num_perm=256)

In [36]:
edges,graph_obj = build_schema(lshensemble, index_dict)

In [37]:
len(graph_obj.edges)

91

In [ ]:
draw_graph(graph_obj)